# OmniIndex
## How to use the omniindex connector for python
examples of each api call are provided in the following notebook cells

written and maintained by: [**@james stanbridge**](mailto:james@omniindex.io)

Contents:
=================
- [OmniIndex](#OmniIndex)
- [V1 API](#V1-API)
    * [get_block_schematic](#get_block_schematic)
    * [run_analytic_query](#run_analytic_query)
    * [generating fake random data](#generating-fake-random-data)
    * [minedata](#minedata)
    * [Appendix](#Appendix)
- [V2 API](#V2-API)


## V1 API

In [ ]:
# mandatory cell
# expects omniindex==0.1.12 minimum
import json
from omniindex.api import OmniIndexClient # this will change in future versions to omniindex.connector
import os
import pandas as pd
import time


if you want to use another method to set your environment variables, you can do so. This is the recommomended method for use with jupyter notebooks

In [ ]:
SERVER = os.environ['OI_API_TEST_SERVER'] # maps to server
API_KEY = os.environ['OI_API_TEST_API_KEY'] # maps to api_key
UNIT_NAME = os.environ['OI_API_TEST_UNIT_NAME']# maps to unit_name
USER = os.environ['OI_API_TEST_USER'] # maps to user
#
BLOCK_TYPE = "Owner"
MASTER_KEY = os.environ['OI_API_MASTER_KEY_DEMO'] # master key for demonstration

APIserver = "https://api.omniindex.xyz/api_v1"

Setting up the client:
as with all the functions, you need to start with a client object = OmniIndexClient()

Parameters
- server (str) – The server to connect to. This is the node server for your blockchain
- api_key (str) – The api key also know as password for your blockchain
- unit_name (str) – The unit name for your blockchain
- user (str) – The user name for your blockchain (paired with the api key)
- block_type (str) – The block type you want to get the schematic for (typically 'Owner' or 'Global')

## get_block_schematic
it's useful to know the structure of the block you're working with. This function will return the block schematic for a given block id

```python
get_block_schematic()
```

In [ ]:
client = OmniIndexClient(SERVER,API_KEY,UNIT_NAME, BLOCK_TYPE, USER, APIserver)
getblock = client.get_block_schematic()
getblock


## run_analytic_query
```python
run_analytic_query(show_protected, query)
```

1. Cell shows what a SQL select * query looks like - note that the show_protected parameter is set to true, which means you will get the protected fields unredacted
2. Cell shows how to SQL select a COUNT of the number of records in the block

In [ ]:
client = OmniIndexClient(SERVER,API_KEY,UNIT_NAME, BLOCK_TYPE, USER, APIserver)
query = client.run_analytic_query("true", "select * from limit 10 ")
query

In [ ]:
client = OmniIndexClient(SERVER,API_KEY,UNIT_NAME, BLOCK_TYPE, USER, APIserver)
query = client.run_analytic_query("true", "select count (*) from ")
print(query)

generating fake-random data for the examples


In [ ]:
from faker import Faker
from faker.providers import internet
""" Creates a fake info for testing purposes
    :param lcid: language codes for localisation
    :type lcid: list
    :return: 
    """

def create_info(lcid: list):
    # Create a Faker object
    fake = Faker(lcid)
    
    # Generate random data
    NAME = fake.name()
    ADDRESS = fake.address()
    INFO = fake.paragraph(nb_sentences=10, variable_nb_sentences=False)
    IPV4 = fake.ipv4()
    CREDIT = fake.credit_card_number()
    CC_EXP = fake.credit_card_expire()
    DOB = str(fake.date_of_birth())
    UUID = fake.uuid4()
    EMOJI = fake.emoji()
    JOB = fake.job()
    PHONE = fake.phone_number()
    USER_AGENT = fake.user_agent()
    
    # Create a JSON object
    data = {
        "nameEncrypt": NAME,
        "addressEncrypt": ADDRESS,
        "infoEncrypt": INFO,
        "ipv4Encrypt": IPV4,
        "creditEncrypt": CREDIT,
        "cc_exp": CC_EXP,
        "dobEncrypt": DOB,
        "uuid": UUID,
        "emoji": EMOJI,
        "job": JOB,
        "phone": PHONE,
        "user_agent": USER_AGENT
    }
    
    return data


    

In [ ]:
data = create_info(['tr_TR', 'en_GB', 'ja_JP'])
data

## minedata

Adds a new block to the blockchain.
- first cell creates a new, random data payload
- second cell adds the data to the blockchain

In [ ]:
# This cell creates a payload (JSON file of records to be added)

# it is recommended practice to always add a timestamp to your data
DATE_ADDED = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())
import datetime

# OmniIndex JSON parser accepts only string values
STAMP = str(datetime.datetime.now().timestamp() * 1000) 

# setup a payload to add to the block
query = create_info(['tr_TR', 'en_GB', 'ja_JP']) # added some localised data just for fun
query["dateAdded"] = DATE_ADDED
payload = json.dumps(query)
print(f"payload = {payload}")

In [ ]:

# create a client object
client = OmniIndexClient(SERVER, API_KEY, UNIT_NAME, BLOCK_TYPE, USER, APIserver)
result = client.post_minedata(MASTER_KEY, payload,)
print(f"This is the return value: {result}")

# APPENDIX
## useful tools etc

Recipes for generating usernames and passwords

In [ ]:
# Generate a ten-character alphanumeric password with at least one lowercase character, at least one uppercase character, and at least three digits:
from faker import Faker


def ten_pass():
    fake = Faker()
    password = fake.password(length=10, special_chars=True, digits=True, upper_case=True, lower_case=True)
    return password

def user_name():
    fake = Faker()
    user_name = fake.user_name()
    return user_name

def pass_key():
    fake = Faker()
    api_key = fake.password(length=32, special_chars=True, digits=True, upper_case=True, lower_case=True)
    return api_key

new_user = user_name()
new_pass = ten_pass()
new_key = pass_key()

print(f"new username: {new_user}")
print(f"new password: {new_pass}")
print(f"new api key: {new_key}")


# V2 API

Basic and pgbc credentials are required to use the V2 API. These are available from the OmniIndex team.



In [ ]:
from omniindex.api import OmniIndex
from omniindex.api_credentials import APICredentials # this is the v2 connector
from omniindex.credentials import BasicCredentials
from omniindex.pgbc_credentials import PGBCCredentials
import psycopg2


credentials = BasicCredentials(username='james', password='james')
print(credentials.are_valid())  # Output: True


credentials = PGBCCredentials(data_source='james@omniindex.io', username='james@omniindex.io', password='james@omniindex.io', server='logs.pgbc.info', port=5434)
print(credentials.are_valid())  # Output: True
print(credentials)  # Output: PGBCCredentials(dataSource=my_db, username=admin, password=password, server=localhost, port=5432)

omniIndex = OmniIndex.with_pgbc_credentials(credentials)
print(omniIndex._credentials.username)  # Output: test-node-server.io

## get_api_key

In [ ]:
import requests

def get_api_key():
    url = "http://api.pgbc.info/cgi-bin/pgbc_api/get_api_key?"
    params = {
        "instance": omniIndex._credentials.dataSource,
        "password": omniIndex._credentials.password,
        "server": omniIndex._credentials.server,
        "port": omniIndex._credentials.port
    }
    
    headers = {
        "Content-Type": "application/json",
        "Referer": "OmniIndex Python Connector",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    }

    try:
        response = requests.post(url, params=params, headers=headers)
        response.raise_for_status()  # Raise an exception for non-200 status codes

        data = response.json()  # Parse JSON response
        api_key = data.get("api_key")

        # Print the API key (replace with your desired usage)
        print("API Key:", api_key)
        return api_key

    except requests.exceptions.RequestException as err:
        print("Error:", err)

# Replace these placeholders with your actual values

api_key = get_api_key()



## get_user_count

In [ ]:
import base64


def get_user_count(input_key):
    if input_key is None:
        raise ValueError("API key must be specified")   
    
    url = "http://api.pgbc.info/cgi-bin/pgbc_api/get_user_count?"
    params = {
        "username": "james",
        "password": "james",
    }
   
    headers = {
        "AUTHORIZATION": f"Basic {input_key}",
        "Referer": "OmniIndex Python Connector",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    }
    
    try:
        print(headers, params)
        response = requests.post(url, params=params, headers=headers)
        response.raise_for_status()

        data = response.text  # Assuming the response is plain text

        # Print the user count (replace with your desired usage)
        print("User Count:", data)
        

    except requests.exceptions.RequestException as err:
        print("Error:", err)

# Replace these placeholders with your actual values
input_key = "UE9TVEdSRVNCQ2luc3RhbmNlPWpP62bhbWVzQG9tbmlpbmRleC5pbyZw12G2cYXNzd29yZD1qY127B2W1lc0BvbW5paW5kZXguP722aW8mc2VydmVyPWxvZ3MucGdiYy5pbmZvJnBvcnQ9NTQzNFBHQkM="
get_user_count(input_key) # Pass the previously obtained API key

## add_block_data

add a single block (row) to the blockchain

> note this code was written for a specific use case, and is not a general purpose function. It is provided as an example of how to use the V2 API
use the api_v2.py and it's classes to add blocks to the blockchain

```python

```

In [ ]:
def add_block_data(input_key):
    url = "http://api.pgbc.info/cgi-bin/pgbc_api//add_block_data?"
    username = "james"
    password = "james"
    ddl = encoded_ddl
    url = url + f"username={username}&password={password}&ddl={ddl}"
    

    headers = {
        "AUTHORIZATION": f"Basic {input_key}",
        "Referer": "OmniIndex Python Connector",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    }

    try:
        response = requests.post(url, headers=headers)
        response.raise_for_status()

        data = response.text  # Assuming the response is plain text

        # Print the response (replace with your desired usage)
        print("Add Block Data Response:", data)

    except requests.exceptions.RequestException as err:
        print("Error:", err)


ddl_value = "INSERT INTO .airport3(airport_id, city, state, nameencrypt) VALUES (10245, 'King Salmon', 'AK', 'King Salmon Airport');"  # The DDL string you want to send
encoded_ddl = ddl_value.replace(" ", "%20")
# Call the function with the API key
add_block_data(input_key)

### Import Function

This still requires a bit of hand hacking to get the data into the right format for the API. The following code will import a csv file into the blockchain. It assumes that the first row of the csv file contains the column names, and that the column names match the block schematic for the block type you are importing into.

```python

In [1]:
import omniindex.api_v2 as api_v2
import csv
def bulk_import_from_csv(api, csv_file_path):
    with open(csv_file_path, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            # Assuming each row contains the necessary data for your DDL statement
            # Adjust the following line according to the structure of your CSV and DDL
            ddl = f"INSERT INTO .airport3(airport_id, city, state, nameencrypt) VALUES ('{row['airport_id']}', '{row['city']}', '{row['state']}', '{row['name']}');"
            response = api.add_block_data(ddl)
            print("Response for row:", row, "is:", response)

# Replace 'your_api_key' with the actual API key etc
your_api_key = "UE9TVEdSRVNCQ2luc3RhbmNlPWpP62bhbWVzQG9tbmlpbmRleC5pbyZw12G2cYXNzd29yZD1qY127B2W1lc0BvbW5paW5kZXguP722aW8mc2VydmVyPWxvZ3MucGdiYy5pbmZvJnBvcnQ9NTQzNFBHQkM="
user_name = "james"
password = "james"

api_instance = api_v2.DatabaseAPI(your_api_key, user_name, password)

# Replace 'path_to_your_csv_file.csv' with the path to your CSV file
bulk_import_from_csv(api_instance, 'airports_bulk.csv')